In [ ]:
import altair as alt
import pandas as pd
# from fuzzywuzzy import fuzz
# from fuzzywuzzy import process
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Adjudicaciones de proyectos
URL = 'https://raw.githubusercontent.com/ANID-GITHUB/Historico-de-Proyectos-Adjudicados/master/BDH_Proyectos.csv'
df_adjudicaciones = pd.read_csv(URL, sep=';',decimal=",")

In [ ]:
filtered_df = df_adjudicaciones[df_adjudicaciones["AÑO_CONCURSO"].isin([2020,2019,2018,
              2017,2016,2015,2014,2013,2012,2011,2010,2009,2008])]

filtered_df2 = filtered_df[filtered_df["MACROZONA MINCIENCIA (segUn ax)"].isin(["CENTRO SUR"])]

filtered_df2

In [ ]:
df_filtered2 = filtered_df2

filtered_df2.columns

def cambiar_nombre(nomb):
    if nomb == "06. OHIGGINS":
        nomb = "O'HIGGINS"
    if nomb == "06. O'HIGGINS":
        nomb = "O'HIGGINS"
    if nomb == "16. 'Ã‘UBLE'":
        nomb = "ÑUBLE"
    if nomb == "08. BIOBIO":
        nomb = "BIO-BIO"
    if nomb == "07. MAULE":
        nomb = "MAULE"

    return nomb

def cambiar_nombre_u(nomb):
    if nomb == "UNIVERSIDAD DEL BIO BIO":
        nomb = "UNIVERSIDAD DEL BIO-BIO"
    else:
        nomb = nomb

    return nomb


filtered_df2["REGION_EJECUCION"] = filtered_df2["REGION_EJECUCION"].apply(cambiar_nombre)

#filtered_df2["INSTITUCION_PRINCIPAL"] = filtered_df2["INSTITUCION_PRINCIPAL"].apply(cambiar_nombre_u)

filtered_df2["REGION_EJECUCION"].unique()

In [ ]:
filtered_df2["REGION_EJECUCION"].unique()

In [ ]:
df_3 = df_filtered2[df_filtered2['SUBDIRECCION'].isin(['PROYECTOS DE INVESTIGACION','INVESTIGACION APLICADA'])].copy()

df_3["INSTITUCION_PRINCIPAL"]


In [ ]:
df_3 = df_filtered2[df_filtered2['SUBDIRECCION'].isin(['PROYECTOS DE INVESTIGACION','INVESTIGACION APLICADA'])].copy()
# TO PREVENT ERRORS
def change_name (inst):
    return inst.replace('UNIV.DE','UNIVERSIDAD DE') 
df_3['INSTITUCION_PRINCIPAL'] = df_3['INSTITUCION_PRINCIPAL'].apply(change_name)
# filtrado solo de universidades
df_3 = df_3[df_3['INSTITUCION_PRINCIPAL'].str.contains("UNIVERSIDAD")]
df_3 = df_3.groupby(['INSTITUCION_PRINCIPAL','REGION_EJECUCION'])\
      .agg({'CODIGO_PROYECTO': 'count'})\
      .rename(columns={'CODIGO_PROYECTO': 'COUNT'})
#tasa auxiliar para el display
#df_3['aux'] = df_3.groupby(level=0).apply(lambda x: x/x.sum())
# la tasa es el porcentaje de participación dentro de la macrozona
df_3['TASA'] = ((df_3.COUNT / df_3.COUNT.sum()))

df_3 = df_3.reset_index()

In [ ]:
df_3

In [ ]:
df_3["INSTITUCION_PRINCIPAL"] = df_3["INSTITUCION_PRINCIPAL"].apply(cambiar_nombre_u)
df_3 = df_3.drop(df_3[df_3['TASA']<0.001].index)
df_3.sort_values("COUNT", ascending=False)

In [ ]:


aux = df_3.groupby(['INSTITUCION_PRINCIPAL']).agg({'COUNT':'sum','TASA':'sum'})
aux = aux.reset_index().sort_values("TASA", axis=0,ascending=False)

text3 = alt.Chart(aux).mark_text(align='left', baseline='middle', dx=10).encode(
        x = alt.X('TASA',sort=None),
        y = alt.Y('INSTITUCION_PRINCIPAL',sort=None),
        text=alt.Text('TASA', format=".2%")
)

chart3 = alt.Chart(df_3.reset_index()).mark_bar().encode(
            x = alt.X('TASA', scale=alt.Scale(domain=[0, 1]),
                axis=alt.Axis(format='%'), sort=None,title=None),
            y = alt.Y('INSTITUCION_PRINCIPAL',sort=None,title=None),
            tooltip = [alt.Tooltip('TASA', title='Tasa', format='.2%'),
                        alt.Tooltip('COUNT', title='Cantidad'),
                        alt.Tooltip('INSTITUCION_PRINCIPAL', title='Institución')],
             color = alt.Color('REGION_EJECUCION',
                      title='Región de ejecución',scale=alt.Scale(scheme='dark2'))

          )


#text3 = chart3.mark_text(align='left', baseline='middle', dx=10).encode(
#          text=alt.Text('TASA', format=".2%")
#        )

# Del gráficos se desprende que la universidad de concepción monopoliza las adjudicaciones
# de la macrozona

# la universidad autónoma es la unica que participa en más de una región
# ñuble no registra adjudicaciones

(text3+chart3).properties(
      title=['Tasa de proyectos de investigación ', 'por institución en la macro zona'])\
    .configure_title(fontSize=20, font='Arial', anchor='middle', color='black')\
    .configure_legend(titleFontSize=14,labelFontSize=12)

## Se superpone el texto en la universidad autónoma, aún no se me ocurre como solucionarlo

In [ ]:
df_4b = df_filtered2
#df_4b = df_4b.groupby(['MACROZONA MINCIENCIA (segUn ax)',"AREA_OCDE"])\
#    .agg({"CODIGO_PROYECTO": "count"}).copy()
df_4b = df_4b.groupby(['AREA_OCDE'])\
    .agg({"CODIGO_PROYECTO": "count"})

df_4b['TASA'] = ((df_4b.CODIGO_PROYECTO / df_4b.CODIGO_PROYECTO.sum()))

df_4b = df_4b.reset_index().rename(columns=({'CODIGO DEL PROYECTO':'COUNT'}))

df_4b.sort_values("CODIGO_PROYECTO",ascending=False)


df_4b = df_4b.drop(df_4b[df_4b['AREA_OCDE']=="NO APLICA"].index)

df_4b

In [ ]:


chart4b = alt.Chart(df_4b).mark_bar().encode(
            x = alt.X('TASA', sort=None, 
                      axis=alt.Axis(format='%')),
            y = alt.Y('AREA_OCDE',sort=None,title=None),
            tooltip = [alt.Tooltip('TASA', title='Tasa', format='.2%'),
                      alt.Tooltip('CODIGO_PROYECTO', title='Cantidad')],
          )
(chart4b).properties(width=450, height=300,
      title=['Tasa de proyectos de investigación', ' por área OCDE Macro zona Centro-Sur',' '])\
      .configure_title( fontSize=20, font='Arial', anchor='middle', color='black')\
      .configure_legend(titleFontSize=14,labelFontSize=12)



In [ ]:
# GRUPO_DE_EVALUACION tiene menos cantidad (están mas agrupados y estandarizados)
# sin embargo se requiere limpiar los datos (puede ser manual o establecer jerarquia 
# con fuzzywuzzy )

#aa = df_adjudicaciones[df_adjudicaciones[]]
df_adjudicaciones.columns

In [ ]:
# esto no me convence, sin embargo concuerdo con luis que para los gráficos 
# anteriores sería mejor considerar DISCIPLINA_DETALLE o algún similar en vez del
# área OCDE (que en realidad es nuestro filtro para la consulta de la mesa 11)
# PD: postulaciones no tiene el dato para filtrar temática y lastimosamente 
#     DISCIPLINA_DETALLE tiene 726 datos distintos (como para graficar) 
#     Respondo estas dudas arriba !!!


adjudicaciones_macrozona = df_adjudicaciones[df_adjudicaciones['MACROZONA MINCIENCIA (segUn ax)']=='CENTRO SUR']

adjudicaciones_macrozona

#Existen 5140 adjudicaciones en la macrozona

adjuducaciones_macrozona = adjudicaciones_macrozona.dropna(subset=['MONTO ADJUDICADO (MILES $)'])

#adjudicaciones_macrozona['MONTO ADJUDICADO (MILES $)'] = adjudicaciones_macrozona['MONTO ADJUDICADO (MILES $)'].str.replace(",", ".").astype(float)

tematica_monto = adjudicaciones_macrozona.groupby(['AREA_OCDE'])[['MONTO ADJUDICADO (MILES $)']].sum()*(1000)/1000000000

tematica_monto = tematica_monto.sort_values('MONTO ADJUDICADO (MILES $)', ascending=False)

tematica_monto['TASA'] = ((tematica_monto['MONTO ADJUDICADO (MILES $)'] / tematica_monto['MONTO ADJUDICADO (MILES $)'].sum()))

tematica_monto = tematica_monto.rename(columns={'MONTO ADJUDICADO (MILES $)' : 'MONTO AJUDICADO MILES MILLONES'})

tematica_monto = tematica_monto.apply(round)

tematica_monto['MONTO AJUDICADO MILES MILLONES'].astype(int)

tematica_monto


